<a href="https://colab.research.google.com/github/katholan/timelaspe_phenotyping_boxes/blob/main/phenotyping_boxes_leaf_reorganization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reorganizing the images from a phenotyping box experiment by leaf instead of by timepoint.
1. Upload your images to a folder on your Google Drive 
2. Change "filepath" to that filepath.
3. Change file extension if necessary.
4. Create a new Google Drive folder where you want your new images to be saved.
5. Change "new_image_dir" to that filepath.
6. Optional: If you would like .gifs of each leaf, create a new Google Drive folder and change "gif_dir" to that filepath.
6. Change the name of your coordinate file in "coords".
7. Optional: Change the number of rows and columns in your sub plots. I normally do 48 timepoints with 6 rows of 8 columns. 
7. Optional: Change the name of "plt.savefig" to what you want your new images to be named.
8. You can run each cell individually or click "Runtime" -> "Run all" to run all cells.
9. Ensure your files are sorted correctly. The order of the files determines the order of the leaves in the output image. If the files are not in the correct HPI order, your leaves will be out of order in your final image. If your files are not sorting correctly, you may need to rename them or add leading zeros to any numbering system.
10. It will likely pause for you to allow access to your files. It will resume automatically after accepting.
11. It will pause for you to choose a .csv file directly from your computer that contains your coordinates to each leaf ROI. After choosing a file, it will continue automatically.

You do not need anything downloaded to your computer to run this script, just an internet connection. The tab to this Google Colab notebook needs to remain open and your computer needs to remain on. For 48 total timepoints with 40 leaves, the entire notebook took about 10 minutes to complete, with the majority of this being the for loop at the very end. After selecting your .csv file, the script does not need any more external input and can be left to run. 

If running successfully, you should begin to see your new images saving to your designated output folder as they are created. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#load necessary packages. some of these are from previous iterations of the script and are not necessary.
import numpy as np
import cv2
import pandas as pd
import matplotlib
import skimage
from matplotlib import pyplot as plt
from skimage import data
from skimage import morphology
from skimage.filters import try_all_threshold
from skimage.filters import threshold_otsu
from skimage.filters import sobel
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, remove_small_objects
from skimage.segmentation import slic
from skimage.color import label2rgb
from scipy import ndimage as ndi
import glob
import timeit
import seaborn as sns
from PIL import Image
from imageio import imread, mimsave

In [ ]:
#connect to your google drive. you will likely need to give permission
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#file paths for your images to be processed and the output folder
filepath = '/content/gdrive/My Drive/raspberry_pi/whithampi2/20220715/'
timepoints=glob.glob(filepath + '*.jpg')
new_image_dir = '/content/gdrive/My Drive/raspberry_pi/whithampi2/20220715/by_leaf'
gif_dir = '/content/gdrive/My Drive/raspberry_pi/whithampi2/20220715/gifs'

In [ ]:
#sort files by timepoint. 
#double check the output to make sure they are in order!
#if they are out of order, make sure you have any necessary leading zeros in your filenames
timepoints.sort()
timepoints

In [ ]:
#allows you to upload a file from your computer. 
#this file will only be available when the notebook is active. It will need to be reloaded each time you restart the notebook.
#I have found this is the easiest way for me to read in a .csv file as uploading it to Drive changes it's extension. 
#uploading from your computer prevents this and allows it to be read into the notebook successfully
from google.colab import files
uploaded = files.upload()

In [ ]:
#read in uploaded csv file and double check contents
#your file should have 5 columns, the name of your ROI (typically leaf number), upper left x coord, upper left y coord, lower right x coord, and lower right y coord.
coords = pd.read_csv("20220715_coords.csv")
print(coords)

In [ ]:
#for loop to generate a single plot for each leaf that includes the images from every timepoint.

for i in range(len(coords)):
  imagelist = []
  for file in timepoints:
    I = Image.open(file)
    imagelist.append(I.crop((coords.iat[i,1],coords.iat[i,2],coords.iat[i,3],coords.iat[i,4])))
  fig = plt.figure(figsize=(45,38))
  for x in range(len(imagelist)):
    plt.subplot(6,8,x+1)
    plt.imshow(imagelist[x])
    plt.xticks([]);
    plt.yticks([]);
    plt.title(x)
  plt.tight_layout()
  plt.savefig(f"{new_image_dir}/leaf_" + str(i) + ".jpg")

In [ ]:
#makes gifs of each leaf. 
#You do not need to run the previous cell to run this cell.
#You can also add this to the previous cell inside the for loop to run them all together.
for i in range(len(coords)):
  imagelist = []
  for file in timepoints:
    I = Image.open(file)
    imagelist.append(np.array(I.crop((coords.iat[i,1],coords.iat[i,2],coords.iat[i,3],coords.iat[i,4]))))
  mimsave(f"{gif_dir}/leaf_" + str(i) + ".gif", imagelist)
